<a href="https://colab.research.google.com/github/G750cloud/20MA573/blob/master/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

data = pd.read_csv('C:/Users/Jing Guo/Desktop/AAPL.csv',index_col=0)
data.head()
data.index.sort_values()
data.index = pd.to_datetime(data.index, format="%Y/%m/%d")
print(data)
data_final = data.drop(['Open','High','Low','Close','Adj Close','Volume','Gross Return','Net Retrun'],axis=1)
print(data_final)
data_final = pd.Series(data_final['log Return'])
print(type(data_final))
print(data_final)

def check_stationarity(data):
    roll_mean = data.rolling(20).mean()
    roll_std = data.rolling(5).std()
    
    fig = plt.figure(figsize=(20,10))
    plt.subplot(211)
    plt.plot(data,color='black',label='Log Return')
    plt.plot(roll_mean,color='red',label='20 days Log Return Mean')
    plt.legend()
    plt.subplot(212)
    plt.plot(roll_std,color='green',label='5 days Log Return Std Deviation')
    
    print('Stationarity Test\n')
    test = adfuller(data,regresults=False)
    print(test)
    result = pd.Series(test[0:4],index=['Test Statustic','p-value','# of lags','# of obs'])
    print(result)
    for k,v in test[4].items():
        print('Critical value at %s: %1.5f' %(k,v))

check_stationarity(data_final)

data_final_diff = data_final-data_final.shift()
data_final_diff.dropna(inplace=True)
check_stationarity(data_final_diff)

data_final_diff2 = data_final-2*data_final.shift()-data_final.shift(2)
data_final_diff2.dropna(inplace=True)
check_stationarity(data_final_diff2)

from statsmodels.tsa.stattools import acf,pacf
data_acf = acf(data_final_diff)
data_pacf = pacf(data_final_diff)

import statsmodels.api as sm
fig1 = plt.figure(figsize=(20,10))
ax1 = fig1.add_subplot(211)
fig1 = sm.graphics.tsa.plot_acf(data_acf,ax=ax1)
ax2 = fig1.add_subplot(212)
fig1 = sm.graphics.tsa.plot_pacf(data_pacf,ax=ax2)

#ARIMA(2,1,0)#
ARIMA = ARIMA(data_final_diff2,(2,0,0))
fit_ARIMA = ARIMA.fit(full_output=True)
predictions = ARIMA.predict(fit_ARIMA.params,start=239,end=242)
print(fit_ARIMA.summary())
predictions
fit_ARIMA.predict(start=245,end=248)